In [28]:
import pandas as pd
import glob
import sys
import os
current_dir = os.getcwd()
sys.path.append(current_dir)  # 루트 디렉터리 경로 추가  # .py
sys.path.append(os.path.dirname(current_dir))  # 상위 디렉터리 경로 추가  # .ipynb
# from src.crawling.nst_item_search import *
from src.preporcess.utils import create_pivot_table, create_aggregate_table, find_missing_dates, fill_missing_dates_and_forward_fill

data_dir = "../data/"

### 시장별 시세

In [29]:
file_path = "processed/item_price_lag_filled.csv"
df = pd.read_csv(data_dir + file_path)
df

,priceDate,item,market,minPrice,avgPrice,maxPrice,avgPrice_lag_1,avgPrice_lag_2,avgPrice_lag_3,avgPrice_lag_7
0,2015-02-25,광어,가락시장,25000.0,25000.0,25000.0,NaN,NaN,NaN,NaN
1,2015-02-26,광어,가락시장,25000.0,25000.0,25000.0,25000.0,NaN,NaN,NaN
2,2015-02-27,광어,가락시장,25000.0,25000.0,25000.0,25000.0,25000.0,NaN,NaN
3,2015-02-28,광어,가락시장,25000.0,25000.0,25000.0,25000.0,25000.0,25000.0,NaN
4,2015-03-01,광어,가락시장,25000.0,25000.0,25000.0,25000.0,25000.0,25000.0,NaN
...,...,...,...,...,...,...,...,...,...,...
163861,2025-01-15,참돔,인천종합연안부두어시장,40000.0,40000.0,40000.0,40000.0,40000.0,40000.0,40000.0
163862,2025-01-16,참돔,인천종합연안부두어시장,40000.0,40000.0,40000.0,40000.0,40000.0,40000.0,40000.0
163863,2025-01-17,참돔,인천종합연안부두어시장,40000.0,40000.0,40000.0,40000.0,40000.0,40000.0,40000.0
163864,2025-01-18,참돔,인천종합연안부두어시장,40000.0,40000.0,40000.0,40000.0,40000.0,40000.0,40000.0


In [30]:
df_pivot = create_pivot_table(
    df=df,
    index_cols=['priceDate', 'market'],  # 기준 컬럼
    columns_col='item',                  # 분류 기준
    values_col='avgPrice'                # 값 컬럼
)
print(len(df_pivot["market"].unique()))
print(find_missing_dates(df_pivot, "priceDate"))
df_pivot

11
DatetimeIndex([], dtype='datetime64[ns]', freq='D')


,priceDate,market,광어_avgPrice,농어_avgPrice,대게_avgPrice,방어_avgPrice,연어_avgPrice,우럭_avgPrice,참돔_avgPrice
0,2015-02-06,노량진 2층,NaN,NaN,32000.0,NaN,NaN,NaN,NaN
1,2015-02-07,노량진 2층,NaN,NaN,32000.0,NaN,NaN,NaN,NaN
2,2015-02-08,노량진 2층,NaN,NaN,32000.0,NaN,NaN,NaN,NaN
3,2015-02-09,노량진 2층,NaN,NaN,32000.0,NaN,NaN,NaN,NaN
4,2015-02-10,노량진 2층,NaN,NaN,32000.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
36616,2025-01-19,노량진 2층,40000.0,36430.0,80720.0,54000.0,6510.0,35000.0,36430.0
36617,2025-01-19,마포농수산물시장,45000.0,45000.0,NaN,NaN,7500.0,35000.0,40000.0
36618,2025-01-19,부산민락어민활어직판장,41000.0,NaN,NaN,NaN,NaN,33340.0,NaN
36619,2025-01-19,소래포구종합어시장,45000.0,45000.0,100000.0,50000.0,8000.0,30000.0,40000.0


In [31]:
# file_path = "prep/item_price_filled_pivot.csv"
# df_pivot.to_csv(data_dir + file_path, index=False)

### 연령별 검색 트렌드

In [32]:
file_list = glob.glob('raw/nst_*.csv')
df_list = [pd.read_csv(data_dir + file_path) for file_path in file_list]
df = pd.concat(df_list, ignore_index=True).drop("idx", axis=1)
df


,age,name,score,date
0,06_12,광어,1.398471,2016-01-01
1,06_12,광어,2.725172,2016-01-02
2,06_12,광어,1.090436,2016-01-03
3,06_12,광어,1.110405,2016-01-04
4,06_12,광어,1.492906,2016-01-05
...,...,...,...,...
253171,60_80,참돔,1.455200,2024-12-27
253172,60_80,참돔,1.132727,2024-12-28
253173,60_80,참돔,1.231929,2024-12-29
253174,60_80,참돔,1.447679,2024-12-30


In [33]:
df_pivot = create_pivot_table(
    df=df,
    index_cols=['date', 'age'],  # 기준 컬럼
    columns_col='name',                  # 분류 기준
    values_col='score'                # 값 컬럼
)
print(len(df_pivot["age"].unique()))
print(find_missing_dates(df_pivot, "date"))
df_pivot

11
DatetimeIndex([], dtype='datetime64[ns]', freq='D')


,date,age,광어_score,농어_score,대게_score,방어_score,연어_score,우럭_score,참돔_score
0,2016-01-01,06_12,1.398471,1.132805,3.010929,1.859701,1.575304,1.132505,1.066252
1,2016-01-01,13_18,1.522796,1.130881,3.265595,1.763233,1.955631,1.509413,1.256832
2,2016-01-01,19_24,1.813454,1.469683,3.654412,2.441575,2.358319,1.701487,1.333213
3,2016-01-01,25_29,1.902568,1.790530,5.434348,3.464911,2.451222,1.926941,1.847831
4,2016-01-01,30_34,1.791911,1.836360,7.978428,3.592016,2.577798,1.845889,1.815447
...,...,...,...,...,...,...,...,...,...
36163,2024-12-31,40_44,1.319904,1.195455,3.635282,2.244991,1.805642,1.307920,1.282782
36164,2024-12-31,45_49,1.284740,1.134216,3.363122,1.954657,1.797459,1.251667,1.232126
36165,2024-12-31,50_54,1.248271,1.207551,3.461566,2.364576,1.872422,1.265440,1.222392
36166,2024-12-31,55_59,1.336911,1.318047,3.886017,3.198864,2.146386,1.417366,1.315056


In [34]:
# file_path = "prep/search_trend_pivot.csv"
# df_pivot.to_csv(data_dir + file_path, index=False)

### 기상정보

In [35]:
file_list = glob.glob('raw/forecast_1525/forecast_*.csv')
df_list = [pd.read_csv(data_dir + file_path, encoding='euc-kr') for file_path in file_list]
df = pd.concat(df_list, ignore_index=True)
df


,지점,일시,풍속(m/s),풍향(deg),GUST풍속(m/s),현지기압(hPa),습도(%),기온(°C),수온(°C),최대파고(m),유의파고(m),평균파고(m),파주기(sec),파향(deg)
0,22105,2020-01-01 00:00,6.4,284.0,9.3,1027.5,48.0,3.2,14.8,3.5,2.0,1.4,8.0,41.0
1,22105,2020-01-01 01:00,6.6,309.0,9.6,1026.7,44.0,3.0,14.8,3.7,1.9,1.4,8.0,45.0
2,22105,2020-01-01 02:00,7.3,292.0,10.7,1026.6,50.0,3.4,14.8,3.4,2.2,1.6,8.0,43.0
3,22105,2020-01-01 03:00,7.7,294.0,11.1,1026.6,47.0,2.8,14.8,3.2,1.9,1.3,5.8,22.0
4,22105,2020-01-01 04:00,7.4,292.0,10.5,1025.9,54.0,3.1,14.8,3.1,2.0,1.4,8.0,45.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
561493,22190,2015-12-31 20:00,4.2,344.0,7.5,1030.1,59.0,8.3,15.6,2.5,1.3,0.9,5.3,190.0
561494,22190,2015-12-31 21:00,2.8,317.0,5.4,1030.2,55.0,8.2,15.7,1.7,1.0,0.7,4.9,197.0
561495,22190,2015-12-31 22:00,6.0,282.0,8.8,1030.2,64.0,8.0,15.8,2.1,1.2,0.8,5.3,190.0
561496,22190,2015-12-31 23:00,6.6,269.0,10.3,1030.1,59.0,7.6,15.8,2.0,1.1,0.8,4.9,170.0


In [36]:
# 기존 데이터프레임의 컬럼명
column_mapping = {
    # '지점': 'station',
    # '일시': 'datetime',
    '풍속(m/s)': '풍속',
    '풍향(deg)': '풍향',
    'GUST풍속(m/s)': 'GUST풍속',
    '현지기압(hPa)': '현지기압',
    '습도(%)': '습도',
    '기온(°C)': '기온',
    '수온(°C)': '수온',
    '최대파고(m)': '최대파고',
    '유의파고(m)': '유의파고',
    '평균파고(m)': '평균파고',
    '파주기(sec)': '파주기',
    '파향(deg)': '파향'
}

# 컬럼명 변경
df_tmp = df.rename(columns=column_mapping)
df_tmp['일시'] = pd.to_datetime(df_tmp['일시']).dt.date
# 결과 확인
print(df_tmp.head())

      지점          일시   풍속     풍향  GUST풍속    현지기압    습도   기온    수온  최대파고  유의파고  \
0  22105  2020-01-01  6.4  284.0     9.3  1027.5  48.0  3.2  14.8   3.5   2.0   
1  22105  2020-01-01  6.6  309.0     9.6  1026.7  44.0  3.0  14.8   3.7   1.9   
2  22105  2020-01-01  7.3  292.0    10.7  1026.6  50.0  3.4  14.8   3.4   2.2   
3  22105  2020-01-01  7.7  294.0    11.1  1026.6  47.0  2.8  14.8   3.2   1.9   
4  22105  2020-01-01  7.4  292.0    10.5  1025.9  54.0  3.1  14.8   3.1   2.0   

   평균파고  파주기    파향  
0   1.4  8.0  41.0  
1   1.4  8.0  45.0  
2   1.6  8.0  43.0  
3   1.3  5.8  22.0  
4   1.4  8.0  45.0  


In [37]:
# 집계 함수 정의
agg_funcs = {
    # 'station': None,                     # 고유 식별자, 집계 제외
    # 'datetime': None,                    # 날짜/시간, 집계 제외
    '풍속': 'mean',                # 평균 풍속
    # '풍향': 'mean',            # 풍향 제외
    'GUST풍속': 'mean',           # 평균 돌풍 풍속
    '현지기압': 'mean',            # 평균 현지 기압
    '습도': 'mean',                  # 평균 습도
    '기온': 'mean',               # 평균 기온
    '수온': 'mean',         # 평균 수온
    '최대파고': 'max',            # 최대 파고
    '유의파고': 'mean',   # 평균 유의 파고
    '평균파고': 'mean',       # 평균 파고
    '파주기': 'mean',               # 평균 파주기
    # '파향': 'mean',            # 파향 제외
}


# 함수 실행
df_agg = create_aggregate_table(df_tmp, index_cols=['지점', '일시'], agg_funcs=agg_funcs)
print(len(df_agg["지점"].unique()))
print(find_missing_dates(df_agg, "일시"))
# print(find_missing_dates(df_agg, "일시"))

10
DatetimeIndex(['2023-04-18'], dtype='datetime64[ns]', freq=None)


In [38]:
# file_path = "prep/forecast_agg.csv"
# df_agg.to_csv(data_dir + file_path, index=False)

### 가락시장 도매가격

In [39]:
file_list = glob.glob('processed/*가격_가락.csv')
df_list = [pd.read_csv(data_dir + file_path) for file_path in file_list]

# 날짜 컬럼 형식 변환
date_col = "날짜"
for i, df in enumerate(df_list):
    df[date_col] = pd.to_datetime(df[date_col])

# 날짜 컬럼 기준으로 병합
df_merged = df_list[0]
for df in df_list[1:]:
    df_merged = pd.merge(df_merged, df, on=date_col, how='outer')
    
df_merged = df_merged.rename(columns={"방어(자연)_대": "방어_대"})
delete_cols = [c for c in df_merged.columns if c[0] == "d"]

df_merged = df_merged.drop(delete_cols, axis=1)
# print(len(df_agg["station"].unique()))
# print(find_missing_dates(df_agg, "date"))
# df_agg
df_merged.columns
df_merged

,날짜,광어_대,광어_중,광어_소,방어_대,우럭_대,우럭_중,우럭_소,참돔_대,참돔_중,참돔_소
0,2018-01-03,21029.0,13823.0,3281.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2018-01-04,20694.0,13933.0,10459.0,15904.0,10831.0,8320.0,6184.0,17115.0,15331.0,12730.0
2,2018-01-05,22167.0,20336.0,16440.0,16004.0,NaN,NaN,NaN,16952.0,15549.0,12728.0
3,2018-01-08,21841.0,21460.0,19800.0,18338.0,9922.0,7651.0,5868.0,17828.0,16646.0,14873.0
4,2018-01-09,20590.0,16115.0,12970.0,18121.0,6000.0,5094.0,4000.0,16410.0,14073.0,9566.0
...,...,...,...,...,...,...,...,...,...,...,...
2142,2024-08-24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16345.0,14400.0,13205.0
2143,2024-08-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18447.0,16950.0,6530.0
2144,2024-09-14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14563.0,7860.0,2216.0
2145,2024-09-16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15486.0,12798.0,8901.0


In [40]:
df_filled = fill_missing_dates_and_forward_fill(df_merged, date_col='날짜')
print(find_missing_dates(df_filled, "날짜"))
df_filled

DatetimeIndex([], dtype='datetime64[ns]', freq='D')


,날짜,광어_대,광어_중,광어_소,방어_대,우럭_대,우럭_중,우럭_소,참돔_대,참돔_중,참돔_소
0,2018-01-03,21029.0,13823.0,3281.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2018-01-04,20694.0,13933.0,10459.0,15904.0,10831.0,8320.0,6184.0,17115.0,15331.0,12730.0
2,2018-01-05,22167.0,20336.0,16440.0,16004.0,10831.0,8320.0,6184.0,16952.0,15549.0,12728.0
3,2018-01-06,22167.0,20336.0,16440.0,16895.0,11367.0,8285.0,5362.0,16802.0,15521.0,12641.0
4,2018-01-07,22167.0,20336.0,16440.0,16895.0,11367.0,8285.0,5362.0,16802.0,15521.0,12641.0
...,...,...,...,...,...,...,...,...,...,...,...
2565,2025-01-11,23000.0,22245.0,21405.0,27978.0,8600.0,8588.0,8500.0,21000.0,20565.0,15000.0
2566,2025-01-12,23000.0,22245.0,21405.0,27978.0,8600.0,8588.0,8500.0,21000.0,20565.0,15000.0
2567,2025-01-13,22564.0,21882.0,21000.0,27978.0,8600.0,8588.0,8500.0,21000.0,20565.0,15000.0
2568,2025-01-14,21764.0,20871.0,20000.0,27978.0,8600.0,8588.0,8500.0,21000.0,20565.0,15000.0


In [41]:
# file_path = "prep/garak_price_filled.csv"
# df_filled.to_csv(data_dir + file_path, index=False)

### 경제지표

In [42]:
file_path = "processed/d경제지표(KOSPI, 원달러, WTI, VIX, 금, 은, MOVE).csv"
df = pd.read_csv(data_dir + file_path)
df

,날짜,KOSPI,USD/KRW,WTI,VIX,Gold,Silver,MOVE,dKOSPI,dUSD/KRW,dWTI,dVIX,dGold,dSilver,dMOVE
0,2018-01-03,2486.350098,1062.540039,61.630001,9.150000,1316.199951,17.184999,47.599998,0.002702,-0.002516,0.020871,-0.063460,0.001903,0.003738,-0.031930
1,2018-01-04,2466.459961,1063.939941,62.009998,9.220000,1319.400024,17.188999,47.380001,-0.008000,0.001318,0.006166,0.007650,0.002431,0.000233,-0.004622
2,2018-01-05,2497.520020,1061.079956,61.439999,9.220000,1320.300049,17.205000,46.070000,0.012593,-0.002688,-0.009192,0.000000,0.000682,0.000931,-0.027649
3,2018-01-08,2513.280029,1060.500000,61.730000,9.520000,1318.599976,17.063999,45.900002,0.006310,-0.000547,0.004720,0.032538,-0.001288,-0.008195,-0.003690
4,2018-01-09,2510.229980,1067.689941,62.959999,10.080000,1311.699951,16.936001,47.290001,-0.001214,0.006780,0.019925,0.058823,-0.005233,-0.007501,0.030283
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1722,2025-01-08,2521.050049,1451.880005,73.320000,17.700001,2664.500000,30.452000,100.760002,0.011617,-0.005187,-0.012525,-0.006734,0.002936,0.000164,-0.019654
1723,2025-01-09,2521.899902,1457.969971,73.919998,17.700001,2683.800049,30.791000,100.760002,0.000337,0.004195,0.008183,0.000000,0.007243,0.011132,0.000000
1724,2025-01-10,2515.780029,1455.229980,76.570000,19.540001,2708.500000,31.091000,96.570000,-0.002427,-0.001879,0.035850,0.103955,0.009203,0.009743,-0.041584
1725,2025-01-13,2489.560059,1472.250000,78.820000,19.190001,2673.500000,30.091000,103.510002,-0.010422,0.011696,0.029385,-0.017912,-0.012922,-0.032164,0.071865


In [82]:
delete_cols = [c for c in df.columns if c[0] == "d"]
df_tmp = df.drop(delete_cols, axis=1)
print(find_missing_dates(df_tmp, "날짜"))

df_filled = fill_missing_dates_and_forward_fill(df_tmp, date_col='날짜')
print(find_missing_dates(df_filled, "날짜"))
df_filled

DatetimeIndex(['2018-01-06', '2018-01-07', '2018-01-13', '2018-01-14',
               '2018-01-20', '2018-01-21', '2018-01-27', '2018-01-28',
               '2018-02-03', '2018-02-04',
               ...
               '2024-12-22', '2024-12-25', '2024-12-28', '2024-12-29',
               '2024-12-31', '2025-01-01', '2025-01-04', '2025-01-05',
               '2025-01-11', '2025-01-12'],
              dtype='datetime64[ns]', length=842, freq=None)
DatetimeIndex([], dtype='datetime64[ns]', freq='D')


,날짜,KOSPI,USD/KRW,WTI,VIX,Gold,Silver,MOVE
0,2018-01-03,2486.350098,1062.540039,61.630001,9.150000,1316.199951,17.184999,47.599998
1,2018-01-04,2466.459961,1063.939941,62.009998,9.220000,1319.400024,17.188999,47.380001
2,2018-01-05,2497.520020,1061.079956,61.439999,9.220000,1320.300049,17.205000,46.070000
3,2018-01-06,2497.520020,1061.079956,61.439999,9.220000,1320.300049,17.205000,46.070000
4,2018-01-07,2497.520020,1061.079956,61.439999,9.220000,1320.300049,17.205000,46.070000
...,...,...,...,...,...,...,...,...
2564,2025-01-10,2515.780029,1455.229980,76.570000,19.540001,2708.500000,31.091000,96.570000
2565,2025-01-11,2515.780029,1455.229980,76.570000,19.540001,2708.500000,31.091000,96.570000
2566,2025-01-12,2515.780029,1455.229980,76.570000,19.540001,2708.500000,31.091000,96.570000
2567,2025-01-13,2489.560059,1472.250000,78.820000,19.190001,2673.500000,30.091000,103.510002


In [46]:
# file_path = "prep/economy_filled.csv"
# df_filled.to_csv(data_dir + file_path, index=False)